<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=160px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Graphs and Networks</h1>
<h1>Lesson IV - Advanced Graph Algorithms</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
import heapq

import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import watermark

%load_ext watermark
%matplotlib inline

Watermark the notebook with current versions of all loaded libraries

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 20.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: fa52d4d7d8878126ee94cab90a757d50d0e3b709

json      : 2.0.9
numpy     : 1.19.2
matplotlib: 3.3.2
watermark : 2.1.0



Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')

We import the previous Graph Class definition

In [4]:
from Graph import *

And add a convenience method to easily add weighted edges

In [5]:
@add_method(Graph)
def add_weighted_edges_from(self, edges):
    # Each edge can have simply a weight of a feature dictionary 
    for edge in edges:
        if isinstance(edge[2], dict):
            self.add_edge(edge[0], edge[1], **edge[2])
        else:
            self.add_edge(edge[0], edge[1], weight=edge[2])

We manually define our undirected graph

In [6]:
G = Graph(directed=False)

In [7]:
G.add_weighted_edges_from(
[
    [0, 1, {"weight":5}],
    [0, 2, 10],
    [0, 4, 2],
    [1, 2, 2],
    [1, 3, 4],
    [2, 3, 7],
    [2, 5, 10],
    [3, 4, 3]
])

Since the graph is undirected, we have double the edges:

In [8]:
G.edges()

[[0, 1, {'weight': 5}],
 [0, 2, {'weight': 10}],
 [0, 4, {'weight': 2}],
 [1, 0, {'weight': 5}],
 [1, 2, {'weight': 2}],
 [1, 3, {'weight': 4}],
 [2, 0, {'weight': 10}],
 [2, 1, {'weight': 2}],
 [2, 3, {'weight': 7}],
 [2, 5, {'weight': 10}],
 [4, 0, {'weight': 2}],
 [4, 3, {'weight': 3}],
 [3, 1, {'weight': 4}],
 [3, 2, {'weight': 7}],
 [3, 4, {'weight': 3}],
 [5, 2, {'weight': 10}]]

## Priority Queue

Our implementation of Dijkstra's algorithm relies on a priority queue. We can easily build one by wrapping around the functionality of the **heapq** standard library module as a priority queue is just a min-heap

In [9]:
class PriorityQueue:
    def __init__(self):
        self.heap = []

    def push(self, node, priority):
        heapq.heappush(self.heap, [priority, node])

    def pop(self, data=True):
        if data:
            return heapq.heappop(self.heap)
        else:
            return heapq.heappop(self.heap)[1]

    def update(self, node, new_priority):
        pos = -1 

        for i, value in enumerate(self.heap):
            priority, node_i = value

            if node_i == node:
                self.heap[i][0] = new_priority
                pos = i
                break

        if pos == -1:
            self.heap.append([new_priority, node])

        heapq.heapify(self.heap)

    def empty(self):
        return len(self.heap) == 0

We test our implementation

In [10]:
queue = PriorityQueue()

In [11]:
queue.push("a", 10)
queue.push("b", 7)
queue.push("c", 6)
queue.push("d", 3)

The contens of the heap are then

In [12]:
queue.heap

[[3, 'd'], [6, 'c'], [7, 'b'], [10, 'a']]

And if we update the priority of one of the elements

In [13]:
queue.update("c", 1)

It becomes

In [14]:
queue.heap

[[1, 'c'], [3, 'd'], [7, 'b'], [10, 'a']]

Finally, we note that we recover a sorted list of items by simply popping them all out of the queue. This is known as Heapsort https://en.wikipedia.org/wiki/Heapsort

In [15]:
print(queue.pop(True))
print(queue.pop(True))
print(queue.pop(True))
print(queue.pop(True))

[1, 'c']
[3, 'd']
[7, 'b']
[10, 'a']


## Dijkstra's algorithm

In [16]:
def dijkstra(G, source):
    N = G.number_of_nodes()
    queue = PriorityQueue()

    dist = {}

    for node in G._nodes.keys():
        dist[node] = [np.inf, []]
    
    dist[source][0] = 0
    dist[source][1].append(source)

    queue.push(source, 0)

    while not queue.empty():
        node_i = queue.pop(False)

        NN = G.neighbours(node_i)

        for node_j in NN:
            weight = G._edges[node_i][node_j]["weight"]
            new_dist = dist[node_i][0] + weight

            if new_dist < dist[node_j][0]:
                dist[node_j][0] = new_dist
                dist[node_j][1] = list(dist[node_i][1])
                dist[node_j][1].append(node_j)

                queue.update(node_j, new_dist)

    return dist

In [17]:
G.edges()

[[0, 1, {'weight': 5}],
 [0, 2, {'weight': 10}],
 [0, 4, {'weight': 2}],
 [1, 0, {'weight': 5}],
 [1, 2, {'weight': 2}],
 [1, 3, {'weight': 4}],
 [2, 0, {'weight': 10}],
 [2, 1, {'weight': 2}],
 [2, 3, {'weight': 7}],
 [2, 5, {'weight': 10}],
 [4, 0, {'weight': 2}],
 [4, 3, {'weight': 3}],
 [3, 1, {'weight': 4}],
 [3, 2, {'weight': 7}],
 [3, 4, {'weight': 3}],
 [5, 2, {'weight': 10}]]

In [18]:
dist = dijkstra(G, 0)
pprint(dist)

{0: [0, [0]],
 1: [5, [0, 1]],
 2: [7, [0, 1, 2]],
 3: [5, [0, 4, 3]],
 4: [2, [0, 4]],
 5: [17, [0, 1, 2, 5]]}


### Path representation

In [19]:
N = G.number_of_nodes()
target = -1*np.ones((N, N), dtype='int')

for node in dist:
    path = dist[node][1]
    target[node, node] = node
    
    for i in range(len(path)-1):
        target[path[i], path[-1]] = path[i+1] 

In [20]:
target

array([[ 0,  1,  1,  4,  4,  1],
       [-1,  1,  2, -1, -1,  2],
       [-1, -1,  2, -1, -1,  5],
       [-1, -1, -1,  3, -1, -1],
       [-1, -1, -1,  3,  4, -1],
       [-1, -1, -1, -1, -1,  5]])

### All shortest paths

In [21]:
for i in range(1, 6):
    dist = dijkstra(G, i)
    
    for node in dist:
        path = dist[node][1]

        for i in range(len(path)-1):
            target[path[i], path[-1]] = path[i+1]

In [22]:
target

array([[0, 1, 1, 4, 4, 1],
       [0, 1, 2, 3, 3, 2],
       [1, 1, 2, 1, 1, 5],
       [4, 1, 1, 3, 4, 1],
       [0, 0, 0, 3, 4, 0],
       [2, 2, 2, 2, 2, 5]])

## Floyd-Warshall Algorithm

In [23]:
def FloydWarshall(G):
    N = G.number_of_nodes()
    dist = np.ones((N, N), dtype='float')*np.inf
    target = -np.ones((N, N), dtype='int')

    for node_i, node_j, w in G.edges():
        weight = w["weight"]
        dist[node_i, node_j] = weight
        target[node_i, node_j] = node_j

    for node_i in G.nodes():
        dist[node_i, node_i] = 0
        target[node_i, node_i] = node_i

    for node_k in range(N):
        for node_i in range(N):
            for node_j in range(N):
                if dist[node_i, node_j] > dist[node_i, node_k] + dist[node_k, node_j]:
                    dist[node_i, node_j] = dist[node_i, node_k] + dist[node_k, node_j]
                    target[node_i, node_j] = target[node_i, node_k]

    return dist, target

In [24]:
G2 = Graph(directed=True)

In [25]:
G2.add_weighted_edges_from(
[
    [1, 0, 4],
    [0, 2, -2],
    [1, 2, 3],
    [2, 3, 2],
    [3, 1, -1],
])

In [26]:
dist, target = FloydWarshall(G2)

In [27]:
pprint(dist)

array([[ 0., -1., -2.,  0.],
       [ 4.,  0.,  2.,  4.],
       [ 5.,  1.,  0.,  2.],
       [ 3., -1.,  1.,  0.]])


In [28]:
pprint(target)

array([[0, 2, 2, 2],
       [0, 1, 0, 0],
       [3, 3, 2, 3],
       [1, 1, 1, 3]])


In [29]:
def path(target, node_i, node_j):
    if target[node_i, node_j] == -1:
        return []

    path = [node_i]

    while node_i != node_j:
        node_i = target[node_i, node_j]
        path.append(node_i)

    return path

In [30]:
print(path(target, 2, 1))
print(path(target, 2, 0))

[2, 3, 1]
[2, 3, 1, 0]


<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>